In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Thyroid_Cancer"
cohort = "GSE82208"

# Input paths
in_trait_dir = "../../input/GEO/Thyroid_Cancer"
in_cohort_dir = "../../input/GEO/Thyroid_Cancer/GSE82208"

# Output paths
out_data_file = "../../output/preprocess/Thyroid_Cancer/GSE82208.csv"
out_gene_data_file = "../../output/preprocess/Thyroid_Cancer/gene_data/GSE82208.csv"
out_clinical_data_file = "../../output/preprocess/Thyroid_Cancer/clinical_data/GSE82208.csv"
json_path = "../../output/preprocess/Thyroid_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression markers differentiating between malignant and benign follicular thyroid tumors"
!Series_summary	"Differential diagnosis between malignant follicular thyroid cancer (FTC) and benign follicular thyroid adenoma (FTA) is a great challenge for even an experienced pathologist and requires special effort. Molecular markers may potentially support a differential diagnosis between FTC and FTA in postoperative specimens. The purpose of this study was to derive molecular support for the differential diagnosis, in the form of a simple multigene mRNA-based classifier that would differentiate between FTC and FTA tissue samples."
!Series_overall_design	"Gene expression profiling was performed in 27 follicular thyroid cancers and 25 follicular thyroid adenomas."
Sample Characteristics Dictionary:
{0: ['Sex: Female', 'Sex: Male', 'Sex: -'], 1: ['age (years): 74', 'age (years): 80', 'age (years): 66', 'age (years): 78', 'age (years): 50', 'age (year

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import json
import os
from typing import Callable, Dict, Optional, Any

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data comparing FTC and FTA
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Thyroid Cancer status is in key 2 (class: FTC/FTA)
trait_row = 2
# Age is in key 1 (age (years): xx)
age_row = 1
# Gender is in key 0 (Sex: Female/Male/-)
gender_row = 0

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> int:
    """Convert thyroid cancer status to binary (0: benign, 1: malignant)"""
    if not value or value == '-':
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # FTC = Follicular Thyroid Cancer (malignant) = 1
    # FTA = Follicular Thyroid Adenoma (benign) = 0
    if value.upper() == 'FTC':
        return 1
    elif value.upper() == 'FTA':
        return 0
    else:
        return None

def convert_age(value: str) -> float:
    """Convert age to continuous value"""
    if not value or value == '-':
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value: str) -> int:
    """Convert gender to binary (0: female, 1: male)"""
    if not value or value == '-':
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.upper() == 'FEMALE':
        return 0
    elif value.upper() == 'MALE':
        return 1
    else:
        return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None
# Conduct initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Note: We'll use clinical_data which should be passed from previous steps
    # We're not loading it from a CSV file as it doesn't exist yet
    
    # Extract clinical features from the previously obtained clinical_data
    # This assumes clinical_data is already loaded in the environment
    try:
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical features
        preview = preview_df(clinical_features)
        print("Preview of clinical features:", preview)
        
        # Save clinical features to CSV
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
    except NameError:
        print("Error: clinical_data not available. Unable to process clinical features.")
        print("Proceeding with metadata validation only.")


Preview of clinical features: {'GSM2186533': [1.0, 74.0, 0.0], 'GSM2186534': [1.0, 80.0, 0.0], 'GSM2186535': [1.0, 66.0, 1.0], 'GSM2186536': [0.0, 78.0, 0.0], 'GSM2186537': [1.0, 74.0, 0.0], 'GSM2186538': [1.0, 50.0, 1.0], 'GSM2186539': [1.0, 61.0, 0.0], 'GSM2186540': [1.0, 72.0, 0.0], 'GSM2186541': [1.0, 80.0, 0.0], 'GSM2186542': [1.0, 61.0, 0.0], 'GSM2186543': [0.0, 38.0, 1.0], 'GSM2186544': [1.0, 66.0, 0.0], 'GSM2186545': [0.0, 47.0, 0.0], 'GSM2186546': [1.0, 69.0, 0.0], 'GSM2186547': [0.0, 44.0, 0.0], 'GSM2186548': [1.0, 72.0, 0.0], 'GSM2186549': [1.0, 66.0, 0.0], 'GSM2186550': [0.0, 34.0, 0.0], 'GSM2186551': [1.0, 39.0, 0.0], 'GSM2186552': [0.0, 23.0, 0.0], 'GSM2186553': [0.0, 14.0, 0.0], 'GSM2186554': [0.0, 42.0, 0.0], 'GSM2186555': [0.0, 29.0, 0.0], 'GSM2186556': [0.0, 43.0, 0.0], 'GSM2186557': [0.0, 31.0, 0.0], 'GSM2186558': [0.0, 58.0, 0.0], 'GSM2186559': [0.0, 52.0, 0.0], 'GSM2186560': [0.0, 60.0, 0.0], 'GSM2186561': [0.0, 69.0, 0.0], 'GSM2186562': [0.0, 60.0, 0.0], 'GSM21865

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
try:
    with gzip.open(matrix_file, 'rt') as file:
        for line in file:
            if "!series_matrix_table_begin" in line:
                found_marker = True
                break
    
    if found_marker:
        print("Found the matrix table marker in the file.")
    else:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        
    # Try to extract gene data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    
    if gene_data.shape[0] == 0:
        print("Warning: Extracted gene data has 0 rows.")
        is_gene_available = False
    else:
        print(f"Gene data shape: {gene_data.shape}")
        # Print the first 20 gene/probe identifiers
        print("First 20 gene/probe identifiers:")
        print(gene_data.index[:20].tolist())
        
except Exception as e:
    print(f"Error extracting gene data: {e}")
    is_gene_available = False
    
    # Try to diagnose the file format
    print("Examining file content to diagnose the issue:")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            for i, line in enumerate(file):
                if i < 10:  # Print first 10 lines to diagnose
                    print(f"Line {i}: {line.strip()[:100]}...")  # Print first 100 chars of each line
                else:
                    break
    except Exception as e2:
        print(f"Error examining file: {e2}")

if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")


SOFT file: ../../input/GEO/Thyroid_Cancer/GSE82208/GSE82208_family.soft.gz
Matrix file: ../../input/GEO/Thyroid_Cancer/GSE82208/GSE82208_series_matrix.txt.gz
Found the matrix table marker in the file.


Gene data shape: (54675, 52)
First 20 gene/probe identifiers:
['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at', '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at', '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at', '1552263_at', '1552264_a_at', '1552266_at']


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers provided, they appear to be Affymetrix probe IDs,
# not standard human gene symbols. These are identifiers like "1007_s_at" which are
# specific to Affymetrix microarray platforms and need to be mapped to standard gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Get a more complete view to understand the annotation structure
print("\nComplete sample of a few rows:")
print(gene_annotation.iloc[:3].to_string())

# Check for columns that might contain gene information
potential_gene_columns = [col for col in gene_annotation.columns if 
                          any(term in col.upper() for term in ["GENE", "SYMBOL", "NAME", "ID"])]
print(f"\nPotential gene-related columns: {potential_gene_columns}")

# Correctly identify the columns for probe ID and gene symbols
gene_id_col = 'ID'  # This is the probe identifier column
gene_symbol_col = 'Gene Symbol'  # This is the gene symbol column

# Create the mapping using the library function
mapping_data = get_gene_mapping(gene_annotation, gene_id_col, gene_symbol_col)

# Print sample of the mapping to confirm
print("\nSample of probe ID to gene symbol mappings:")
print(mapping_data.head(10))

# Check the size of the mapping data
print(f"\nTotal number of probe-to-gene mappings: {len(mapping_data)}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 

### Step 6: Gene Identifier Mapping

In [7]:
# Make sure we have the gene expression data and gene mapping available
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 1. Load gene expression data
gene_expression = get_genetic_data(matrix_file)
print(f"Gene expression data shape: {gene_expression.shape}")

# 2. Get gene annotation
gene_annotation = get_gene_annotation(soft_file)

# 3. Create gene mapping
# From previous analysis, we observed:
# - The gene expression data has probe IDs like '1007_s_at' as index
# - The gene annotation has 'ID' column for probe IDs and 'Gene Symbol' column for gene names
prob_col = 'ID'  # Column containing probe identifiers
gene_col = 'Gene Symbol'  # Column containing gene symbols
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)

print(f"Gene mapping shape: {mapping_df.shape}")
print("Sample of mapping data:")
print(mapping_df.head())

# 4. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_expression, mapping_df)
print(f"After mapping, gene expression data shape: {gene_data.shape}")
print("Sample of gene expression data (first 5 genes, first 5 samples):")
print(gene_data.iloc[:5, :5])

# Save the gene expression data to a file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene expression data shape: (54675, 52)


Gene mapping shape: (45782, 2)
Sample of mapping data:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A
After mapping, gene expression data shape: (21278, 52)
Sample of gene expression data (first 5 genes, first 5 samples):
          GSM2186533  GSM2186534  GSM2186535  GSM2186536  GSM2186537
Gene                                                                
A1BG        3.448852    3.270417    2.646647    4.187510    2.979016
A1BG-AS1    2.257473    2.257473    2.257473    2.257473    2.257473
A1CF        4.621940    4.514946    4.514946    4.514946    4.514946
A2M        12.614231   13.090670   12.192458   11.504684   12.318218
A2M-AS1     3.884622    4.393652    2.257473    3.929452    3.692808


Gene expression data saved to ../../output/preprocess/Thyroid_Cancer/gene_data/GSE82208.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
try:
    # Load the gene data that was saved in the previous step
    gene_data = pd.read_csv(out_gene_data_file, index_col=0)
    print(f"Loaded gene data shape before normalization: {gene_data.shape}")
    
    # Apply normalization to gene symbols
    gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {gene_data.shape}")
    
    # Save the normalized gene data
    gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    is_gene_available = True
except Exception as e:
    print(f"Error loading or normalizing gene data: {e}")
    is_gene_available = False

# 2. Load clinical data with correct row indices
try:
    # Load the original clinical data (from Step 1)
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Extract clinical features with correct indices from Step 2
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=2,  # Correct value from Step 2
        convert_trait=convert_trait,
        age_row=1,
        convert_age=convert_age,
        gender_row=0,
        convert_gender=convert_gender
    )
    
    print(f"Extracted clinical data shape: {clinical_features.shape}")
    print("Preview of clinical data (first 5 samples):")
    print(clinical_features.iloc[:, :5])
    
    # Save the properly extracted clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    is_trait_available = True
except Exception as e:
    print(f"Error extracting clinical data: {e}")
    is_trait_available = False

# 3. Link clinical and genetic data if both are available
if is_trait_available and is_gene_available:
    try:
        # Debug the column names to ensure they match
        print(f"Gene data columns (first 5): {gene_data.columns[:5].tolist()}")
        print(f"Clinical data columns (first 5): {clinical_features.columns[:5].tolist()}")
        
        # Check for common sample IDs
        common_samples = set(gene_data.columns).intersection(clinical_features.columns)
        print(f"Found {len(common_samples)} common samples between gene and clinical data")
        
        if len(common_samples) > 0:
            # Link the clinical and genetic data
            linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
            print(f"Initial linked data shape: {linked_data.shape}")
            
            # Debug the trait values before handling missing values
            print("Preview of linked data (first 5 rows, first 5 columns):")
            print(linked_data.iloc[:5, :5])
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            if linked_data.shape[0] > 0:
                # Check for bias in trait and demographic features
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                
                # Validate the data quality and save cohort info
                note = "Dataset contains gene expression data from follicular thyroid cancer and follicular thyroid adenoma tissue samples."
                is_usable = validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=is_biased,
                    df=linked_data,
                    note=note
                )
                
                # Save the linked data if it's usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Data not usable for the trait study - not saving final linked data.")
            else:
                print("After handling missing values, no samples remain.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="No valid samples after handling missing values."
                )
        else:
            print("No common samples found between gene expression and clinical data.")
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,
                df=pd.DataFrame(),
                note="No common samples between gene expression and clinical data."
            )
    except Exception as e:
        print(f"Error linking or processing data: {e}")
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,  # Assume biased if there's an error
            df=pd.DataFrame(),  # Empty dataframe for metadata
            note=f"Error in data processing: {str(e)}"
        )
else:
    # We can't proceed with linking if either trait or gene data is missing
    print("Cannot proceed with data linking due to missing trait or gene data.")
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=True,  # Data is unusable if we're missing components
        df=pd.DataFrame(),  # Empty dataframe for metadata
        note="Missing essential data components for linking (trait data or gene expression data)."
    )

Loaded gene data shape before normalization: (21278, 52)
Gene data shape after normalization: (19845, 52)


Normalized gene data saved to ../../output/preprocess/Thyroid_Cancer/gene_data/GSE82208.csv
Extracted clinical data shape: (3, 52)
Preview of clinical data (first 5 samples):
                GSM2186533  GSM2186534  GSM2186535  GSM2186536  GSM2186537
Thyroid_Cancer         1.0         1.0         1.0         0.0         1.0
Age                   74.0        80.0        66.0        78.0        74.0
Gender                 0.0         0.0         1.0         0.0         0.0
Clinical data saved to ../../output/preprocess/Thyroid_Cancer/clinical_data/GSE82208.csv
Gene data columns (first 5): ['GSM2186533', 'GSM2186534', 'GSM2186535', 'GSM2186536', 'GSM2186537']
Clinical data columns (first 5): ['GSM2186533', 'GSM2186534', 'GSM2186535', 'GSM2186536', 'GSM2186537']
Found 52 common samples between gene and clinical data
Initial linked data shape: (52, 19848)
Preview of linked data (first 5 rows, first 5 columns):
            Thyroid_Cancer   Age  Gender      A1BG  A1BG-AS1
GSM2186533           

Linked data shape after handling missing values: (52, 19848)
For the feature 'Thyroid_Cancer', the least common label is '0.0' with 25 occurrences. This represents 48.08% of the dataset.
The distribution of the feature 'Thyroid_Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 42.75
  50% (Median): 60.0
  75%: 69.0
Min: 14.0
Max: 80.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 11 occurrences. This represents 21.15% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Thyroid_Cancer/GSE82208.csv
